In [1]:
import os
import sys
from pathlib import Path
root = str(Path(os.path.abspath(os.path.curdir)).parent)
sys.path.insert(1, root)

import numpy as np
import cv2
from constant import TEST_FRAGMENTS_PATH

In [2]:
pred_images = []
fragments = [f for f in os.listdir(TEST_FRAGMENTS_PATH) if f != '.DS_Store']

for fragment in fragments:
    mask_path = os.path.join(TEST_FRAGMENTS_PATH, fragment, 'mask.png')
    x, y = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).shape
    pred_image = np.random.rand(x, y)
    pred_images.append(pred_image)

In [3]:
def rle(output, threshold):
    print('1')
    flat_img = output.flatten()
    print('2')
    flat_img = np.where(flat_img > threshold, 1, 0).astype(np.uint8)
    print('3')
    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    print('4')
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    print('5')
    starts_ix = np.where(starts)[0] + 2
    print('6')
    ends_ix = np.where(ends)[0] + 2
    print('7')
    lengths = ends_ix - starts_ix
    print('8')
    return ' '.join(map(str, sum(zip(starts_ix, lengths), ())))

In [ ]:
pred = rle(pred_images[0], threshold=0.8)